# Basic

## Setup

In [1]:
#| default_exp basic

In [1]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *

from fh_commons.core import *
from fh_commons.static import *

import json
import pandas as pd

In [2]:
hdrs = bootstrap_hdrs()

In [3]:
show(*hdrs)

In [4]:
#| eval: False
from ngrok_token import *
url = start_ngrok(token)

In [5]:
url

'https://49d1-34-207-219-130.ngrok-free.app'

In [6]:
app,rt = fast_app(pico=False, live=True,hdrs=hdrs)
server = JupyUvi(app)

## Routes

### @rt

Both get and post method, function name is the route's name

In [ ]:
@rt
def test():
    return P('hi')

### @rt('/sth')

If function name is get or post, then would be either of the method.

If function name is something else, would not change the route, but specify the route name

In [9]:
@rt('/something')
def get(): return P('hi')

With params:

In [10]:
@rt('/something/{data}')
def get(data:str): return P(data)

In [11]:
htmx(url,'/something/sdf')

Use function to indicate route's name

In [12]:
@rt('/something')
def sdfsdf(req): 
    return P(req.url_for('sdfsdf'))

With route's name

In [13]:
@rt('/something',name='aaa')
def get(req): 
    return P(req.url_for('aaa'))

### @app.get/post

Similar to @rt, the function name indicates route

In [20]:
@app.get
def asdf():
    return P('hi')

### @app.get('/something')

In [15]:
@app.get('/user/{nm}')
def get_nm(nm:str): return f"Good day to you, {nm}!"

## Request

### req.url

Get current page's url

In [7]:
@rt
def sdf(req):
    return P(req.url)

In [8]:
htmx(url,'/sdf')

### req.url_for

Get original 

In [9]:
@rt
def sdf(req):
    return P(req.url_for('sdf'))

In [10]:
htmx(url,'/sdf')

With param:

In [11]:
@rt("/{data}")
def this(req,data:str):
    return req.url_for('this',data=data)

In [12]:
htmx(url,'/ssss')

## Toasts

toasts options: "info", "success", "warning", "error"

In [13]:
setup_toasts(app)

In [14]:
@rt
def test():
    return A('click to trigger toasts',hx_get=tt,target_id='content'),Div(id='content')

@rt
def tt(sess):
    return add_toast(sess, "Please upload a csv or excel file", "error")

In [15]:
htmx(url,'/test')

## Download file through a link

In [ ]:
@app.get
def download_example():
    return FileResponse('data/example.csv')

@rt
def test(req):
    return A('Download Example File', 
             href=req.url_for('download_example'), # for dynamic route
             download='example.csv', # download file name
            )

In [ ]:
htmx(url,'/test')

## File upload & receive

### Upload csv/excel file 

In [23]:
#| export
def get_file_input(label_text,id,**kwargs):
    return Form(
        Label(label_text, fr=id, cls='form-label'),
        Input(type='file', id=id, cls='form-control'),
        **kwargs
    )

In [46]:
get_file_input('Select csv/excel for upload',id='sdf')

<div>
<form enctype="multipart/form-data"><label for="sdf" class="form-label">Select csv/excel for upload</label>    <input type="file" id="sdf" class="form-control" name="sdf">
</form><script>if (window.htmx) htmx.process(document.body)</script></div>


### Receive file

In [ ]:
#| export
def bytes2df(bytes_data,file_type):
    if file_type == "csv":
        data_io = StringIO(bytes_data.decode("utf-8"))
        return pd.read_csv(data_io)
    elif file_type == "excel":
        data_io = BytesIO(bytes_data)
        return pd.read_excel(data_io)

### Example

In [47]:
@rt
def file_upload(req):
    add = get_file_input(
        'Upload your csv or excel',
        id='myFile',post=preview, target_id='content',hx_trigger='change')
    
    example_file = A(Small('Download Example File'), href=req.url_for('download_example'),download='example.csv')
    
    return Group(add,example_file), Div(id='content')

@app.get
def download_example():
    return FileResponse('data/example.csv')

@app.post
async def preview(sess, myFile:UploadFile):
    filename = myFile.filename

    if filename.endswith('.csv'):
        file_type = 'csv'
    elif filename.endswith(('.xls', '.xlsx')):
        file_type = 'excel'
    else:
        return add_toast(sess, "Please upload a csv or excel file", "error")
        
    # Data as byte string
    bytes_data = await myFile.read()
    df = bytes2df(bytes_data,file_type)
    
    if len(df)>100_000:
        return add_toast(sess, "Exceed 100,000 lines, please use python api for large file", "warning")
    
    return df2html(df.head())

In [48]:
htmx(url,'/file_upload')

## Pagination

In [8]:
def get_pagination(page,total_pages,routes):
    "Bootstrap-styled pagination with integrated page jump form"
    return Nav(
        Ul(
            Li(A("Previous", href=f"{routes}?page={page-1}" if page > 1 else "#", 
                 cls="page-link" + (" disabled" if page <= 1 else "")), 
               cls="page-item"),
            Li(Span(f"Page {page} of {total_pages}", cls="page-link"), cls="page-item active"),
            Li(A("Next", href=f"{routes}?page={page+1}" if page < total_pages else "#", 
                 cls="page-link" + (" disabled" if page >= total_pages else "")), 
               cls="page-item"),
            Li(
                Form(
                    Input(type="number", name="page", placeholder="Page", min="1", max=str(total_pages), 
                          cls="form-control", style="width: 100px; display: inline-block;"),
                    Button("Go", type="submit", cls="btn btn-primary"),
                    cls="d-flex align-items-center m-2 g-2",
                    method="get",
                    action=routes
                ),
                cls="page-item"
            ),
            cls="pagination justify-content-center align-items-center"
        ),
        aria_label="Page navigation"
    )

Suppose I want a pagination bar for page 3 of 20

In [9]:
%%s
get_pagination(3,20,'#')

The above pagination can be combined with this function:

In [10]:
#| export
def get_page_index(current_page, total_items, items_per_page=12):
    "Get start index and end index given the current page number"
    
    total_pages = ceil(total_items / items_per_page) #ceil returns integer number, ceil(4.2) --> 5
    
    page = max(1, min(current_page, total_pages))  # Ensure page is within valid range
    
    start_index = (current_page - 1) * items_per_page
    end_index = start_index + items_per_page
    
    return start_index, end_index,total_pages

Suppose I have a list of 100 items, and I want each page have 12 items, so I need to calculate start index and end index given the current page (e.g., 2)

In [11]:
get_page_index(2,100,12)

(12, 24, 9)

Example for a pagination:

In [120]:
protein_list = [str(i) for i in range(100)]

In [138]:
@rt
def test(req, page: int = 1):

    # get item index
    start_idx, end_idx,total_pages = get_page_index(page, len(protein_list))

    # get items
    current_proteins = protein_list[start_idx:end_idx]
    show_proteins = P(','.join(current_proteins))

    # get dynamic route
    this_routes =req.url_for('test')

    # get pagination item
    pagination = get_pagination(page,total_pages,routes=this_routes)

    return show_proteins, pagination,this_routes

In [139]:
htmx(url,'/test')

## Modal

### Modal content

In [20]:
#| export
def modal_content(title, *args):
    return Div(
        Div(
            Div(
                H1(title, cls='modal-title fs-5'),
                Button(type='button', data_bs_dismiss='modal', aria_label='Close', cls='btn-close'),
                cls='modal-header'
            ),
            Div(*args, cls='modal-body'),
            cls='modal-content'
        ),
        cls='modal-dialog modal-xl'
    )

In [21]:
%%s
modal_content('Modal content','sdfsdf')

It can be combined with the below to search and trigger modal

### Use form button to trigger modal

In [73]:
#| export
def modal_form(*form_content,post, btn_text='Search',cls=None,target_id='modals-window'):
    "Reference: https://htmx.org/examples/modal-bootstrap/"
    modal_button = Form(*form_content, # Input(type='text',id='ssss')
                        Div(
                            Button(btn_text,cls='btn btn-primary py-3'),
                            data_bs_toggle='modal', 
                            data_bs_target=f'#{target_id}'),
                        post=post, 
                        target_id=target_id, 
                        hx_trigger='submit', 
                        cls=cls)

    modal_content =Div(
        Div(
            Div(cls='modal-content'),
            role='document',
            cls='modal-dialog modal-lg modal-dialog-centered'
        ),
        id=target_id,
        style='display: none',
        aria_hidden='false',
        tabindex='-1',
        cls='modal modal-blur fade'
    )
    return modal_button, modal_content

In [79]:
@rt
def page():
    modal_send = modal_form(Input(type='text',id='input_text'),post=modal)
    return modal_send

@rt
def modal(input_text:str):
    return modal_content('Modal title',input_text)

In [80]:
htmx(url,'/page')

### Use anchor link to trigger modal

In [78]:
#| export
def modal_link(text,post,cls=None,target_id='modals-window'):
    "Reference: https://htmx.org/examples/modal-bootstrap/"
    modal_button = A(text,
                     data_bs_toggle='modal', 
                     data_bs_target=f'#{target_id}',
                     post=post, 
                     target_id=target_id, 
                     cls=cls)

    modal_content =Div(
        Div(
            Div(cls='modal-content'),
            role='document',
            cls='modal-dialog modal-lg modal-dialog-centered'
        ),
        id=target_id,
        style='display: none',
        aria_hidden='false',
        tabindex='-1',
        cls='modal modal-blur fade'
    )
    return modal_button, modal_content

In [65]:
@rt
def page():
    modal_send = modal_link('click here',post=modal)
    return modal_send

@rt
def modal():
    return modal_content('Modal title','sdf')

In [66]:
htmx(url,'/page')

## Input select

In [59]:
def get_input(label_text,id,cls=None, type='text',**kwargs):
    return Div(
        Input(type=type, id=id, cls='form-control',placeholder=label_text,**kwargs),
        Label(label_text, fr=id),
        cls=f'form-floating {cls}')

In [63]:
%%s
get_input('Gene ID',id='aa')

In [64]:
def get_input_list(label_text, input_list, id, cls=None, type='text', **kwargs):
    input_div = Div(
        Input(type=type, id=id, cls='form-control', placeholder=label_text, autocomplete="off", **kwargs),
        Label(label_text, fr=id),
        Div(
            *[Div(item, cls='dropdown-item', data_value=item) for item in input_list],
            id=f'{id}-dropdown',
            cls='dropdown-menu'
        ),
        cls=f'form-floating position-relative {cls}'
    )

    script = Script(f"""
    document.addEventListener('DOMContentLoaded', function() {{
        setupAutocomplete('#{id}', '#{id}-dropdown');
    }});
    """)

    return input_div, script

In [69]:
%%s
get_input_list('Select item', ['a','b','c','d'],id='select')

## TextArea

In [85]:
%%s
Textarea(id='s', placeholder='sdf', cls='form-control',rows=10)

## Select (default is first)

In [ ]:
#| export
def get_select(label_text,option_list,id,cls=None, **kwargs):
    "The first item in the option_list is the default"
    return Div(
        Select(
            Option(option_list[0], value=option_list[0],selected=True),
            *[Option(i, value=i) for i in option_list[1:]],
            id = id,
            cls='form-select',
            **kwargs
        ),
        Label(label_text, fr=id),
        cls=f'form-floating {cls}')


## Select

In [87]:
def get_select_simple(label_text, select_list,id,cls=None):
    return Div(Select(
        Option(label_text, selected=True,disabled=True),
        *[Option(i,value=i) for i in select_list],
        cls='form-select',
        id = id), cls=cls)

In [90]:
%%s
get_select_simple('select one', ['a','b','c'],id='ssdf')

## Select multiple

In [91]:

def get_select_simple_multiple(label_text, select_list,id,cls=None):
    return Div(Select(
        Option(label_text, selected=True,disabled=True),
        *[Option(i,value=i) for i in select_list],
        cls='form-select',
        multiple=True,
        id = id), cls=cls)

In [93]:
%%s
get_select_simple_multiple('select one', ['a','b','c'],id='ssss')

## Spinner

In [47]:
def get_spinner(id, cls='d-flex justify-content-center'):
    return Div(Div(
        Span('Loading...', cls='visually-hidden'),
        role='status',
        cls='htmx-indicator spinner-border',
        id=id,
    ),cls=cls)

In [49]:
%%s
get_spinner(id='sdf')

Example:

In [ ]:
@app.post
def example():
    ...
    send_select = Form(
            Hidden(value=df.to_json(),id='upload_df'),
            Div(select_list,button,cls='row g-2 align-items-center'),
            post=calculate,
            target_id='result',
            hx_indicator='#spinner',
        )
    
    spinner=get_spinner(id='spinner')
    
    return send_select, spinner, ...


## End

In [1]:
#| hide
import nbdev; nbdev.nbdev_export()